[Reference](https://kernc.github.io/backtesting.py/doc/examples/Quick%20Start%20User%20Guide.html)

In [1]:
!pip install backtesting

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/175.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/175.5 kB ? eta -:--:--
     -- ------------------------------------- 10.2/175.5 kB ? eta -:--:--
     -- ------------------------------------- 10.2/175.5 kB ? eta -:--:--
     ------ ------------------------------ 30.7/175.5 kB 217.9 kB/s eta 0:00:01
     ------------ ------------------------ 61.4/175.5 kB 297.7 kB/s eta 0:00:01
     ------------------- ----------------- 92.2/175.5 kB 374.1 kB/s eta 0:00:01
     ------------------------- ---------- 122.9/175.5 kB 450.6 kB/s eta 0:00:01
     --------------------------------- -- 163.8/175.5 kB 517.2 kB/s eta 0:00:01
     ------------------------------------ 175.5/175.5 kB 480.4 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd


In [3]:
# Example OHLC daily data for Google Inc.
from backtesting.test import GOOG

GOOG.tail()


C:\Users\xz035\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\xz035\AppData\Roaming\Python\Python312\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
C:\Users\xz035\AppData\Roaming\Python\Python312\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.ht

,Open,High,Low,Close,Volume
2013-02-25,802.3,808.41,790.49,790.77,2303900
2013-02-26,795.0,795.95,784.40,790.13,2202500
2013-02-27,794.8,804.75,791.11,799.78,2026100
2013-02-28,801.1,806.99,801.03,801.20,2265800
2013-03-01,797.8,807.14,796.15,806.19,2175400


In [4]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [5]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [7]:
%%script echo

    def next(self):
        if (self.sma1[-2] < self.sma2[-2] and
                self.sma1[-1] > self.sma2[-1]):
            self.position.close()
            self.buy()

        elif (self.sma1[-2] > self.sma2[-2] and    # Ugh!
              self.sma1[-1] < self.sma2[-1]):
            self.position.close()
            self.sell()

Couldn't find program: 'echo'


In [8]:
from backtesting import Backtest

bt = Backtest(GOOG, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   97.067039
Equity Final [$]                  68221.96986
Equity Peak [$]                   68991.21986
Return [%]                         582.219699
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   25.266427
Volatility (Ann.) [%]               38.383008
Sharpe Ratio                         0.658271
Sortino Ratio                        1.288779
Calmar Ratio                         0.763748
Max. Drawdown [%]                  -33.082172
Avg. Drawdown [%]                   -5.581506
Max. Drawdown Duration      688 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                   94
Win Rate [%]                        54.255319
Best Trade [%]                       57.11931
Worst Trade [%]                    -16.629898
Avg. Trade [%]                    

In [9]:
stats._strategy

<Strategy SmaCross>

In [10]:
bt.plot()

C:\Users\xz035\AppData\Roaming\Python\Python312\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\xz035\AppData\Roaming\Python\Python312\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\xz035\AppData\Roaming\Python\Python312\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\xz035\AppData\Roaming\Python\Python312\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1309', ...)

In [11]:
stats.tail()

Expectancy [%]                                             2.606294
SQN                                                        1.990216
_strategy                                                  SmaCross
_equity_curve                      Equity  DrawdownPct DrawdownD...
_trades               Size  EntryBar  ExitBar  EntryPrice  ExitP...
dtype: object

In [12]:
stats['_equity_curve']

,Equity,DrawdownPct,DrawdownDuration
2004-08-19,10000.00000,0.000000,NaT
2004-08-20,10000.00000,0.000000,NaT
2004-08-23,10000.00000,0.000000,NaT
2004-08-24,10000.00000,0.000000,NaT
2004-08-25,10000.00000,0.000000,NaT
...,...,...,...
2013-02-25,67624.41986,0.019811,NaT
2013-02-26,67570.01986,0.020600,NaT
2013-02-27,68390.26986,0.008711,NaT
2013-02-28,68510.96986,0.006961,NaT


In [13]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-59,63,75,168.68196,179.13,-616.43436,-0.061939,2004-11-17,2004-12-06,19 days
1,52,75,85,179.48826,182.00,130.61048,0.013994,2004-12-06,2004-12-20,14 days
2,-52,85,88,181.63600,187.45,-302.32800,-0.032009,2004-12-20,2004-12-23,3 days
3,49,88,110,187.82490,179.27,-419.19010,-0.045547,2004-12-23,2005-01-26,34 days
4,-49,110,119,178.91146,196.96,-884.37846,-0.100880,2005-01-26,2005-02-08,13 days
...,...,...,...,...,...,...,...,...,...,...
89,82,1947,1960,611.57070,588.72,-1873.75740,-0.037364,2012-05-11,2012-05-31,20 days
90,-83,1960,1983,587.54256,580.01,625.20248,0.012820,2012-05-31,2012-07-03,33 days
91,85,1983,2059,581.17002,705.58,10574.84830,0.214068,2012-07-03,2012-10-19,108 days
92,-85,2059,2087,704.16884,702.24,163.95140,0.002739,2012-10-19,2012-12-03,45 days
